### должен быть установлен LibreOffice (желательно на линуксе)
необходим для конвертации старого док формата в новый

In [11]:
import pandas as pd
from docx import Document
import re
import os
from contextlib import contextmanager
import subprocess
import zipfile

In [7]:
districts = [
            "Адмиралтейский",
            "Василеостровский",
            "Выборгский",
            "Калининский",
            "Кировский",
            "Колпинский",
            "Красногвардейский",
            "Красносельский",
            "Кронштадтский",
            "Курортный",
            "Московский",
            "Невский",
            "Петроградский",
            "Петродворцовый",
            "Приморский",
            "Пушкинский",
            "Фрунзенский",
            "Центральный"
            ]
tables_per_dist = [1, 2, 1, 1, 2, 2, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2]
dist_to_tables = dict.fromkeys(districts, [])

In [3]:
distr_to_number_of_tables = dict(zip(districts, tables_per_dist))
distr_to_number_of_tables

{'Адмиралтейский': 1,
 'Василеостровский': 2,
 'Выборгский': 1,
 'Калининский': 1,
 'Кировский': 2,
 'Колпинский': 2,
 'Красногвардейский': 3,
 'Красносельский': 1,
 'Кронштадтский': 1,
 'Курортный': 1,
 'Московский': 1,
 'Невский': 1,
 'Петроградский': 2,
 'Петродворцовый': 1,
 'Приморский': 1,
 'Пушкинский': 1,
 'Фрунзенский': 1,
 'Центральный': 2}

In [4]:
which_distr = [[dist] * n for dist, n in zip(districts, tables_per_dist)]
which_distr = [item for sublist in which_distr for item in sublist]
assert(len(which_distr) == sum(tables_per_dist))

```python

 { дата `ddmmyyyy`: {район: { газ: [токсичность] || единств. значение если одна табличка в районе }}}
 # список потому что мб несколько таблиц
    
```

In [28]:
DateDistrGas = {} # структура преставлена выше

def ratio_parser(ratio):
    '''
    парсилка для значения
    '''
    start_menee = re.compile(r'^\s*менее', re.I)

    if ',' in ratio:
        ratio = re.sub(',', '.', ratio)
    if not re.search('\d+\.\d+', ratio):
        ratio = None
    elif re.search(start_menee, ratio):
        ratio = 0.05
    else:
        ratio = float(ratio)
    return ratio

@contextmanager
def docx_table_parser(dest):
    try:
        cwd = os.getcwd()
        os.chdir(dest)
        
        
        # на самом деле можно и просто os.listdir() - здесь уже не важно
        docxs = sorted(os.listdir(), key=os.path.getmtime)
        
        n_docxs = len(docxs)
        bar_tick = n_docxs // 100
        c, ticks = 0, 0
        
        for doc in docxs:
            
            date = doc[:-5]
            try:
                ref = open(doc,"rb")
                office_doc = Document(ref)
            except (ValueError, zipfile.BadZipFile):
                print(f"Corrupted file name {doc}")
                subprocess.call(['lowriter', '--convert-to', 'docx', doc])
                assert(zipfile.is_zipfile(doc))
                office_doc = Document(doc)

            DateDistrGas[date] = {} # ключ без расширения
            
            n_dist_tables = 0
            i_current_dist_table = 0
            
            for i_table in range(len(office_doc.tables)):
                'имя района'
                distr_name = which_distr[i_table]
                'число таблиц по имени района (1, 2 или 3)'
                n_dist_tables = distr_to_number_of_tables[distr_name]

                if distr_name not in DateDistrGas[date]: # если района с таким именем еще нет в такую дату
                    'добавляем пустой список на'
                    DateDistrGas[date][distr_name] = {}
                    
                    if distr_name not in districts:
                        print(distr_name)

                n = 1 # пропускаем шапку таблицы
                while True:
                    row = office_doc.tables[i_table].row_cells(n)
                    if not row:
                        break

                    gas, ratio = row[0].text.lower().strip(), row[1].text
                    ratio = ratio_parser(ratio)
                    if gas not in DateDistrGas[date][distr_name]:
                        if n_dist_tables < 2:
                            DateDistrGas[date][distr_name][gas] = ratio
                        else:
                            DateDistrGas[date][distr_name][gas] = [ratio]
                    else:
                        try:
                            DateDistrGas[date][distr_name][gas].append(ratio)
                        except AttributeError:
                            'какая-то дичь в файле'
                            print(DateDistrGas[date].keys(), gas, date)
                            
                    n += 1

                if i_current_dist_table < n_dist_tables - 1:
                    i_current_dist_table += 1
                else:
                    i_current_dist_table = 0
            
            c += 1
            if not c % bar_tick:
                ticks += 1
                if not ticks % 5:
                    print(f'{ticks}% complete')
            
        yield
    finally:
        os.chdir(cwd)

In [29]:
with docx_table_parser('download'):
    pass

5% complete
10% complete
15% complete
20% complete
25% complete
30% complete
35% complete
40% complete
45% complete
50% complete
55% complete
60% complete
65% complete
70% complete
75% complete
80% complete
85% complete
90% complete
95% complete
100% complete
dict_keys(['Адмиралтейский']) № стан-ции 01102018


In [33]:
DateDistrGas['01102018'] #??? должно быть неадекватный файл

{'Адмиралтейский': {'№ стан-ции': None,
  '1': None,
  '2': None,
  '3': None,
  '4': None,
  '5': None,
  '7': None,
  '8': None,
  '9': None,
  '11': None,
  '13': None,
  '14': None,
  '15': None,
  '16': None,
  '17': None,
  '18': None,
  '19': None,
  '20': None,
  '21': None,
  '23': None,
  '24': None,
  '25': None},
 'Василеостровский': {'№ стан-ции': [None, None, None, None],
  '1': [None, None],
  '2': [None, None],
  '3': [None, None],
  '4': [None, None],
  '5': [None, None],
  '7': [None, None],
  '8': [None, None],
  '9': [None, None],
  '11': [None, None],
  '13': [None, None],
  '14': [None, None],
  '15': [None, None],
  '16': [None, None],
  '17': [None, None],
  '18': [None, None],
  '19': [None, None],
  '20': [None, None],
  '21': [None, None],
  '23': [None, None],
  '24': [None, None],
  '25': [None, None]}}

In [17]:
len(DateDistrGas.keys())

1131

вызов данных по всем районам за какое-то число
например 10 июня 2018 г.

In [18]:
DateDistrGas['10062018']

{'Адмиралтейский': {'оксид углерода': 0.1,
  'оксид азота': 0.1,
  'диоксид азота': 0.7,
  'диоксид серы': 0.05},
 'Василеостровский': {'оксид углерода': [0.1, 0.1],
  'оксид азота': [0.1, 0.1],
  'диоксид азота': [0.5, 0.7],
  'взвешенные частицы  pm2,5': [0.1],
  'взвешенные частицы рм10': [0.1]},
 'Выборгский': {'оксид углерода': 0.1,
  'оксид азота': 0.1,
  'диоксид азота': 0.7,
  'взвешенные частицы рм10': 0.1},
 'Калининский': {'оксид углерода': 0.1,
  'оксид азота': 0.05,
  'диоксид азота': 0.8,
  'диоксид серы': 0.05,
  'взвешенные частицы  pm10': 0.1},
 'Кировский': {'оксид углерода': [0.1, 0.1],
  'оксид азота': [0.05, 0.1],
  'диоксид азота': [0.4, 0.6],
  'диоксид серы': [0.05, 0.1],
  'взвешенные частицы  pm10': [0.1, 0.2]},
 'Колпинский': {'оксид углерода': [0.1, 0.1],
  'оксид азота': [0.2, 0.1],
  'диоксид азота': [0.6, 0.6],
  'диоксид серы     ': [0.1, 0.1],
  'взвешенные частицы  pm10': [0.1, 0.1]},
 'Красногвардейский': {'оксид углерода': [0.1, 0.05, 0.05],
  'оксид

## Парсилка текста для любого документа

In [4]:
d = Document('download/20102018.docx')

In [9]:
def docx_text_parser(docx_doc):
    full_doc = ""
    for par in d.paragraphs:
        full_doc += fr"{par.text}"
        full_doc += " "
    
    output = []
    for i in range(len(districts)):
        try:
            found = re.search(fr'{districts[i]}\W*район\W*(Средние.*?)(?=район)', full_doc).group(0)
            output.append(found)
        except AttributeError:
            if districts[i] == "Центральный":
                found = re.search(fr"Центральный\s*район.*", full_doc).group()
                output.append(found)
    return output

In [10]:
docx_text_parser(d)

['Адмиралтейский район  Средние концентрации загрязняющих веществ, выраженные в единицах среднесуточных предельно допустимых концентраций (ПДКс.с.) по данным автоматической станции мониторинга загрязнения атмосферного воздуха, расположенной по адресу: Московский пр., дом 19 (станции АСМ-АВ №10), составляют:  Радиационная обстановка находится в пределах нормы  Василеостровский ',
 'Василеостровский район  Средние концентрации загрязняющих веществ, выраженные в единицах среднесуточных предельно допустимых концентраций (ПДКс.с.) по данным автоматической станции мониторинга загрязнения атмосферного воздуха, расположенной по адресу: В.О.Средний пр., дом 74 (станции АСМ-АВ №24), составляют: Радиационная обстановка находится в пределах нормы  Выборгский ',
 'Выборгский район  Средние концентрации загрязняющих веществ, выраженные в единицах среднесуточных предельно допустимых концентраций (ПДКс.с.) по данным автоматической станции мониторинга загрязнения атмосферного воздуха, расположенной по 